<a href="https://colab.research.google.com/github/gagusti/algorithm-opt-viu/blob/main/Algoritmos_AG3_Guillermo_Agusti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG3 - Actividad Guiada 3

Guillermo Agusti